**Feature Engineering Recap**

Opium Sown: Have number of sown hectares for each district in each province for each year between 2010 - 2020, except for part of Bamyan, Day Kundi, Farah, Faryab, Ghazni, and Ghor, where we have 2008-2018 data. Pending further investigation, we will code these provinces with zeros. All fields marked '-' or 'p-f' (poppy-free) in the CSV are replaced by zeros. Data is of numerical type. All NaN values are set to zero.

Soil Data: Dropped the WRB Codes column because it contained numerous inaccuracies as an artefact of the scraping process and because it was collinear with the Soil Type column. Soil Type column was categorical and has been turned into dummy variables with one hot encoding. All other data was of numerical type. Province name is broadcast for each row of soil sample information. All fields marked (-)(-) in the CSV are replaced by zeros. Dataframe only contains the estimates of sown areas. All margin of error information is removed from the dataframe. All NaN values are set to zero.

BIG CAVEAT FOR SOIL DATA: We do not have unique soil data for all 34 provinces. We only have soil data for 9 provinces. I cannot find any up to date soil data for all the provinces - only PDF maps from 2011 that shows topsoil texture distribution (source: Afghan Geodesy and Cartography Head Office, conforms to United Nations Afghanistan Regions 3958.1 R3, June 2011), and qualitative assessments from the UNODC Opium Yields reports. So we are going to subjectively broadcast the soil data from each of the 9 provinces to the closest provinces whose topsoil texture distributions closely resemble each others per the 2011 maps. To get a single soil feature reading for each province, we will consider the area of each soil type in each province as the "weight" vector and multiply it to the respective chemical measurement of that soil type, and take the sum of each multiplied column, kind of like a weighted average. We will divide each soil feature column by the total area of that province to normalize. And so we will have with subjective soil quality metrics for each province. But these broadcast readings should be subsituted with unique soil sample results as soon as the data becomes available.

Temperature and Precipitation Data: All data is transposed such that the years are columns and months are rows. Province name is broadcast for each row of climatological information. All NaN values are set to zero. Data is of numerical type.



In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
tract_directory = "/content/ML-Climate-Final-Project-Template/data"
opium_sown = None
soil_data = None
temp_data = None
precip_data = None
climate_data = None
for filename in os.listdir(tract_directory):
  fn = tract_directory + '/' + filename
  if "Opium" in fn:
    section_frame = pd.read_csv (fn, header=0)
    section_frame.dropna(how='all', inplace=True)
    section_frame = section_frame.loc[:, ~section_frame.columns.str.contains('^Unnamed')]
    section_frame = section_frame[(section_frame['Province'].str.contains("Total")==True) | (section_frame['District'].str.contains("Total")==True)]
    section_frame.fillna('', inplace=True)
    section_frame["combo"] = section_frame["Province"] + section_frame["District"]
    section_frame['new province'] = section_frame['combo'].map(lambda tot_str: tot_str.partition('Total')[0])
    Y_sown = section_frame.drop(['Province', 'District', 'combo', 'new province'], axis=1)
    Y_sown.replace(regex={'[^0-9]': 0}, inplace=True)
    Y_sown["Province"] = section_frame['new province']
    Y_sown.replace(regex={'Sari  Pul': 'Sar-e-Pul'}, inplace=True)
    Y_sown.replace(regex={'\s+$': ''}, inplace=True)
    if opium_sown is None:
      opium_sown = Y_sown
    else:
      opium_sown = pd.concat([opium_sown, Y_sown]).reset_index(drop=True)
      opium_sown.fillna(0, inplace=True)    

  if "Soil" in fn:
    section_frame = pd.read_csv (fn, header=0)
    section_frame.set_axis(['WRB_Code', 'Soil_Type', 'Area', 'Sand_Perc', 'Clay_Perc', 'OM_Perc', 'pH_Water', 'EC', 'Tot_N_ppm', 'P_ppm', 'K_ton_per_ha', 'S_ppm', 'CaCO_ton_per_ha3'], axis=1, inplace=True)
    section_frame.drop(['WRB_Code', 'Soil_Type'], axis=1, inplace=True)
    section_frame.replace(regex={'\(+.*': '', '±.*': '', ' ±.*': '', ' \(±.*': '', '\(-\)\(-\)': 0,  '\(±.*': ''}, inplace=True)
    section_frame.replace(regex={'[^0-9.]': ''}, inplace=True)
    section_frame.replace(r'^\s*$', np.NaN, regex=True, inplace=True)
    section_frame.fillna(0, inplace=True)
    section_frame.dropna(how='all', inplace=True)
    section_frame = section_frame.astype('float')
    
    province = filename.split('_')[0]
    if province == 'Balkh':
      similar_provs = ['Balkh', 'Kunduz', 'Jawzjan', 'Samangan', 'Sar-e-Pul', 'Faryab']
    elif province == 'Bamyan':
      similar_provs = ['Bamyan', 'Day Kundi', 'Ghor', 'Ghazni']
    elif province == 'Hirat':
      similar_provs = ['Hirat', 'Badghis', 'Farah']
    elif province == 'Kabul':
      similar_provs = ['Kabul', 'Wardak', 'Logar', 'Kapisa', 'Parwan']
    elif province == 'Kandahar':
      similar_provs = ['Kandahar', 'Uruzgan', 'Zabul']
    elif province == 'Khost':
      similar_provs = ['Khost', 'Paktika', 'Paktya']
    elif province == 'Nangarhar':
      similar_provs = ['Nangarhar', 'Kunar', 'Laghman']
    elif province == 'Nimroz':
      similar_provs = ['Nimroz', 'Hilmand']
    elif province == 'Takhar':
      similar_provs = ['Takhar', 'Badakhshan', 'Baghlan', 'Panjsher', 'Nuristan']

    X_soil = pd.DataFrame(similar_provs, columns=['Province'])
    for col_name in section_frame.columns.values.tolist():
      if col_name != 'Area':
        X_soil[col_name] = pd.Series(section_frame['Area'] * section_frame[col_name]).sum()
        X_soil[col_name]  = X_soil[col_name] / pd.Series(section_frame['Area']).sum()

    if soil_data is None:
      soil_data = X_soil
    else:
      soil_data = pd.concat([soil_data, X_soil]).reset_index(drop=True)

  if ("pr" in fn) or ("tas" in fn):
    info = pd.read_csv(fn, skiprows=2, nrows=0)
    section_frame = pd.read_csv (fn, skiprows=3)
    section_frame.rename(columns={'Unnamed: 0':'Years'}, inplace=True )
    section_frame.drop(section_frame[section_frame['Years'] < 2010].index, inplace = True)
    section_frame.insert(loc=0, column='Province', value=info.columns[1])
    section_frame.replace(regex={'Daykundi': 'Day Kundi'}, inplace=True)
    section_frame.fillna(0, inplace=True)
    if "pr" in fn:
      section_frame.columns = section_frame.columns[:2].union('mean_precip_' + section_frame.columns[2:])
      if precip_data is None:
        precip_data = section_frame
      else:
        precip_data = pd.concat([precip_data, section_frame]).reset_index(drop=True)

    elif "tas" in fn:
      section_frame.columns = section_frame.columns[:2].union('mean_temp_' + section_frame.columns[2:])
      if temp_data is None:
        temp_data = section_frame
      else:
        temp_data = pd.concat([temp_data, section_frame]).reset_index(drop=True)

climate_data = precip_data.merge(temp_data, on=['Province', 'Years'])

province_list = [
                 'Balkh', 
                 'Kunduz', 
                 'Jawzjan', 
                 'Samangan', 
                 'Sar-e-Pul', 
                 'Faryab', 
                 'Bamyan', 
                 'Day Kundi', 
                 'Ghor', 
                 'Ghazni', 
                 'Hirat', 
                 'Badghis', 
                 'Farah', 
                 'Kabul', 
                 'Wardak', 
                 'Logar', 
                 'Kapisa', 
                 'Parwan', 
                 'Kandahar', 
                 'Uruzgan', 
                 'Zabul', 
                 'Khost', 
                 'Paktika', 
                 'Paktya', 
                 'Nangarhar', 
                 'Kunar', 
                 'Laghman',
                 'Nimroz',
                 'Hilmand',
                 'Takhar', 
                 'Badakhshan', 
                 'Baghlan', 
                 'Panjsher', 
                 'Nuristan'
                 ]
opium_sown = opium_sown[opium_sown['Province'].isin(province_list)]
opium_sown = opium_sown.melt(id_vars=['Province'], var_name="Years", value_name="Hectares_Sown")

print("Features Compiled. Dataframes:")
print("Opium Sown")
print(opium_sown)
print("Soil Data")
print(soil_data)
print("Climate Data")
print(climate_data)

Features Compiled. Dataframes:
Opium Sown
      Province Years Hectares_Sown
0     Panjsher  2010             0
1       Parwan  2010             0
2     Samangan  2010             0
3    Sar-e-Pul  2010             0
4       Takhar  2010             0
..         ...   ...           ...
369  Nangarhar  2020          2225
370     Nimroz  2020          2931
371   Nuristan  2020             0
372    Paktika  2020             0
373     Paktya  2020             0

[374 rows x 3 columns]
Soil Data
      Province  Sand_Perc  Clay_Perc   OM_Perc  pH_Water        EC  \
0       Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   
1    Day Kundi  60.661439   4.905354  0.877024  7.993465  0.310647   
2         Ghor  60.661439   4.905354  0.877024  7.993465  0.310647   
3       Ghazni  60.661439   4.905354  0.877024  7.993465  0.310647   
4        Khost  54.983806  16.427164  1.567991  8.151691  0.197600   
5      Paktika  54.983806  16.427164  1.567991  8.151691  0.197600   
6       Paktya

In [3]:
X_set = soil_data.merge(climate_data, on='Province', how='outer')
print(X_set)

    Province  Sand_Perc  Clay_Perc   OM_Perc  pH_Water        EC  Tot_N_ppm  \
0     Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   0.000000   
1     Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   0.000000   
2     Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   0.000000   
3     Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   0.000000   
4     Bamyan  60.661439   4.905354  0.877024  7.993465  0.310647   0.000000   
..       ...        ...        ...       ...       ...       ...        ...   
369    Zabul  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112   
370    Zabul  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112   
371    Zabul  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112   
372    Zabul  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112   
373    Zabul  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112   

         P_ppm  K_ton_per_ha      S_ppm  ...  mean_

**Random Forest Benchmark Regressor**

For each province, for each year, the X dataset is the local soil features, and the mean temperature and mean precipitation in the 12 months of the precending year, while the Y is the number of hectares sown in the current year. We have climatological data from 2010 through 2020 (11 years). We will use 34 provinces * 10 years from 2010 to 2019 = 340 datapoints in total for training and testing. Once we have fine-tuned our benchmark, we will use the 2020 climatological features (and existing soil featuers) to predict the number of hectares of opium sown in 2021, and compare our prediction against the UNODC report that will come out later in the year.

In [4]:
X_set['Years_to_match'] = X_set['Years'] + 1
X_set['primary_key'] = X_set['Province'] + '_' + X_set['Years_to_match'].astype(str)
X_set.drop(['Years_to_match'], axis=1)

X_set.set_index('primary_key', inplace=True)
X_set.drop(X_set[X_set['Years'] > 2019].index, inplace = True)
X_set.drop(['Years', 'Province'], axis=1, inplace=True)

opium_sown['primary_key'] = opium_sown['Province'] + '_' + opium_sown['Years'].astype(str)
opium_sown.set_index('primary_key', inplace=True)

opium_sown.drop(opium_sown[opium_sown['Years'].astype(int) < 2011].index, inplace = True)
opium_sown.drop(['Years', 'Province'], axis=1, inplace=True)

# Need to ensure X and Y datapoint match up by order of entry in respective df
dataset_all = pd.merge(X_set, opium_sown, left_index=True, right_index=True).reset_index(drop=True)

opium_sown = dataset_all[['Hectares_Sown']].astype(float)
X_set = dataset_all.drop(['Hectares_Sown', 'Years_to_match'], axis=1)

print("opium_sown:")
print(opium_sown)
print("X_set:")
print(X_set)

opium_sown:
     Hectares_Sown
0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
..             ...
335            0.0
336            0.0
337            0.0
338          183.0
339          408.0

[340 rows x 1 columns]
X_set:
     Sand_Perc  Clay_Perc   OM_Perc  pH_Water        EC  Tot_N_ppm      P_ppm  \
0    60.661439   4.905354  0.877024  7.993465  0.310647   0.000000  29.910692   
1    60.661439   4.905354  0.877024  7.993465  0.310647   0.000000  29.910692   
2    60.661439   4.905354  0.877024  7.993465  0.310647   0.000000  29.910692   
3    60.661439   4.905354  0.877024  7.993465  0.310647   0.000000  29.910692   
4    60.661439   4.905354  0.877024  7.993465  0.310647   0.000000  29.910692   
..         ...        ...       ...       ...       ...        ...        ...   
335  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112  32.179072   
336  62.086269  14.910295  0.840974  7.929618  0.742861  36.879112  32.179072   


In [5]:
import statsmodels.api as sm
### @Mohar add constant to X matrix 
X = sm.add_constant(X_set)

model1 = sm.OLS(opium_sown, X).fit()
predictions = model1.predict(X)
model1.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Hectares_Sown   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     2.682
Date:                Sun, 27 Mar 2022   Prob (F-statistic):           6.88e-06
Time:                        22:15:12   Log-Likelihood:                -3322.8
No. Observations:                 340   AIC:                             6712.
Df Residuals:                     307   BIC:                             6838.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              627.1202    501.847      1.250      0.212    -360.375    1614.615
Sand_Perc         -300.4897    288.461     -1.042      0.298    -868.101     267.121
Clay_Perc        -1752.3123   1445.305     -1.212      0.226   -4596.269    1091.645
OM_Perc          -3397.1351   3476.543     -0.977      0.329   -1.02e+04    3443.733
pH_Water          5481.2209   4640.084      1.181      0.238   -3649.170    1.46e+04
EC               -5410.3726   5479.383     -0.987      0.324   -1.62e+04    5371.527
Tot_N_ppm          189.7310    142.116      1.335      0.183     -89.914     469.376
P_ppm               33.1047    100.651      0.329      0.742    -164.949     231.159
K_ton_per_ha     -3474.3836   2418.976     -1.436      0.152   -8234.255    1285.488
S_ppm             -529.1886    440.699     -1.201      0.231   -1396.362     337.985
CaCO_ton_per_ha3     7.7605     10.846      0.716      0.475     -13.580      29.102
mean_precip_Apr    -10.4434     20.154     -0.518      0.605     -50.101      29.215
mean_precip_Aug      8.5194     13.210      0.645      0.519     -17.474      34.513
mean_precip_Dec     -6.2186     14.403     -0.432      0.666     -34.560      22.122
mean_precip_Feb     24.9705     18.184      1.373      0.171     -10.810      60.751
mean_precip_Jan    -48.1912     32.169     -1.498      0.135    -111.491      15.109
mean_precip_Jul     21.2440     48.961      0.434      0.665     -75.098     117.586
mean_precip_Jun     36.1430     52.727      0.685      0.494     -67.610     139.896
mean_precip_Mar     33.0764     36.608      0.904      0.367     -38.958     105.110
mean_precip_May    -73.2541     39.171     -1.870      0.062    -150.332       3.824
mean_precip_Nov    -50.5547     35.661     -1.418      0.157    -120.725      19.616
mean_precip_Oct     10.8946     21.951      0.496      0.620     -32.298      54.087
mean_precip_Sep    -16.9486     27.402     -0.619      0.537     -70.868      36.971
mean_temp_Apr      633.3347    470.855      1.345      0.180    -293.176    1559.846
mean_temp_Aug      -24.1247    322.288     -0.075      0.940    -658.298     610.049
mean_temp_Dec       91.4897    312.053      0.293      0.770    -522.545     705.524
mean_temp_Feb      273.6929    486.959      0.562      0.574    -684.507    1231.893
mean_temp_Jan      818.6320    409.935      1.997      0.047      11.995    1625.269
mean_temp_Jul     -330.1213    602.376     -0.548      0.584   -1515.429     855.187
mean_temp_Jun     1082.5130    646.914      1.673      0.095    -190.434    2355.460
mean_temp_Mar    -1720.2351    813.866     -2.114      0.035   -3321.697    -118.774
mean_temp_May      -80.0682    442.635     -0.181      0.857    -951.051     790.915
mean_temp_Nov      584.7714    508.686      1.150      0.251    -416.181    1585.723
mean_temp_Oct     -237.

In [6]:
column_set = list(X.columns)
column_set.remove('const')

# Trying to figure out which independent variables are important
# by running the regression minus one column at a time
# and looking at the drop in adjusted R^2 value, if any
# which would indicate that the dropped column was explaining
# at least some percentage of the variation in the Y (opium sown)

for colname in column_set:
  X_subset = X.loc[:, X.columns != colname]
  modelnew = sm.OLS(opium_sown, X_subset).fit()
  predictions = modelnew.predict(X_subset)
  print("Removed Column:", colname)
  print(modelnew.summary())

Removed Column: Sand_Perc
                            OLS Regression Results                            
Dep. Variable:          Hectares_Sown   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     2.682
Date:                Sun, 27 Mar 2022   Prob (F-statistic):           6.88e-06
Time:                        22:15:12   Log-Likelihood:                -3322.8
No. Observations:                 340   AIC:                             6712.
Df Residuals:                     307   BIC:                             6838.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const         